### **notebook_plotter() by D. Principe (principe@space.mit.edu)**

#### This [jupyter notebook](https://cxc.cfa.harvard.edu/ciao/faq/jupyter.html) tutorial uses the [sherpa](https://cxc.cfa.harvard.edu/sherpa/) contributed tool 'notebook_plotter' and [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/index.html) to interactively view X-ray spectral models and data. Model parameters such as absorbing column density (nH) or plasma temperature (kT) can be adjusted using a slider bar and the plotted models (unconvolved and/or convolved) will update simultaneously.  This tool is useful for exploring parameter space for fitting astrophysical models to data and for proposal planning purposes. While this tool was built with Chandra data in mind it should be compatible with any sherpa loaded dataset and their corresponding response files. 

#### This tutorial assumes the reader already has some basic knowledge about [fitting spectral models to data](https://cxc.cfa.harvard.edu/sherpa/threads/pha_intro/) and the use of [Sherpa](https://cxc.cfa.harvard.edu/sherpa/). 

#### This notebook tutorial includes the following three examples:

<b>

- EXAMPLE 1: Navigating the plotter and examining an unconvolved spectral model.  

- EXAMPLE 2: Loading and fitting a spectrum while visually inspecting how the model parameters may influence the overall fit.

- EXAMPLE 3 - Simulating a Chandra spectrum for a proposal with help from notebook_plotter().
</b>

##### Current limitations:  

- Interactive plots are limited to a single model expression and/or dataset id. You cannot plot two different models or datasets in the same notebook_plotter window. However, you can switch between separate loaded datasets using the notebook_plotter ‘dataset_id’ parameter.  

- Functionality of the widget slider bars depend on a reasonable range of parameter min/max values. Default min/max values for some model parameters are physically unreasonable and must be constrained before plotting with notebook_plotter.  


### **USAGE INSTRUCTIONS**



#### Once [CIAO 4.17](https://cxc.cfa.harvard.edu/ciao/download/) or later is initialized in your terminal window, you can run this tutorial notebook on your own machine using the following commands:

unix% mkdir notebooks  
unix% cp $ASCDS_INSTALL/share/sherpa/notebooks/*ipynb notebooks/  
unix% jupyter lab --notebook-dir notebooks  

#### You can also load the notebook_plotter() tool into any **[jupyter notebook](https://cxc.cfa.harvard.edu/ciao/faq/jupyter.html)** running sherpa and CIAO 4.17 or later using the following commands inside a notebook:

from sherpa.astro.ui import *  
from sherpa_contrib.notebook_plotter import notebook_plotter  

#### For this tutorial, we start by loading the necessary modules for sherpa and notebook_plotter functionality.

In [ ]:
from sherpa.astro.ui import *
from sherpa_contrib.notebook_plotter import notebook_plotter

import numpy as np
import matplotlib.pyplot as plt    

#### We can use help(notebook_plotter) to learn more about the function and its parameters.

In [ ]:
help(notebook_plotter)

#### This notebook includes several functional examples of how to use notebook_plotter(). As such, it's a good idea to run sherpa's [clean()](https://cxc.cfa.harvard.edu/sherpa/ahelp/clean.html) function before any example in case you want to re-run this example after running later examples in this notebook.

In [ ]:
clean()

## **EXAMPLE 1: Navigating the plotter and examining an unconvolved spectral model**

#### Let's use sherpa to generate a spectral model and plot it with notebook_plotter().The model here is an absorbed, one-temperature thermal plasma typical of coronal emission from a bright, young pre-main sequence star.

In [ ]:
test_model = xstbabs.ma1 * (xsapec.p1)
test_model #we can print the model parameters out to the notebook 

#### Looking over the model parameters (above), we'll want to constrain what will be plotted using physically-appropriate parameter limits. For example, xspec xstbabs model has a maxmimum nH value of 1,000,000 (1E28 atoms $\mathrm{cm^{-2}}$) and that value is too high for the molecular environment surrounding nearby young stars. 

In [ ]:
ma1.nH.min = 0.01
ma1.nH.max = 40
p1.redshift.min = 0

#### Now let's open this model in the notebook_plotter() by simply calling it as a variable to the notebook_plotter function. A description of the plot is provided in the next jupyter notebook cell.

In [ ]:
notebook_plotter(test_model)

#### *In the above interactive window you can see the basic notebook interface with the plotting options on the left and the model parameters on the right. Each parameter of notebook_plotter is controlled using an ipywidget and there are several different ipywidgets to interact with. You can activate 'sliding widgets' in one of three ways: (a) by clicking on the blue circle and dragging the parameter left or right, (b) clicking on the line where you want the slider to go, or (c) clicking the value to the right of the widget and typing in a new value. 'Float text' widgets are boxes with a value displayed. You can click in the box and enter your own value or you can hover to the right of the box and click up or down to change the parameter by one widget unit. The 'dropdown' widgets can be selected by clicking them and picking an option from the dropdown menu. Note that the 'autoscale' option overrides the x and/or y axis limits unless 'autoscale' is set to 'none'.*

#### *The previous cell's interactive plot was created by calling only the model with notebook_plotter. However, this plotting tool can also be opened in a specific state by calling the function with the parameters listed in the plotting options using the same syntax. In the below example, we can open the same model and initialize the plotting window parameters via the command line instead of interactively.*

In [ ]:
notebook_plotter(test_model, log_axis = 'ylog', autoscale = 'y', xlim_low = 0.3, xlim_high =10, figsize_y = 3, plot_type = 'model_unconvolved')

#### **It's important to understand that the test_model paramater values (e.g., nH, kT, etc.) WILL be updated in your sherpa session with the widget values selected in the interactive plot after running notebook_plotter()**. This can be useful when trying to fit models to data but dangerous if you are not aware that model values are being updated.   

#### Since model parameters are updated when you interact with the model widgets, it is not recommended to go back to the previous cell's (old) interactive window without first reloading it. Note how updating the model parameters in the above plot does not change the model parameters from the first interactive plot until it is reloaded.

In [ ]:
test_model()

#### *Here we can see that the sherpa model values have been updated to the same as those chosen with the widgets in the interactive plot.*

## **EXAMPLE 2: Loading and fitting a spectrum while visually inspecting how the model parameters may influence the overall fit.**

#### This example is meant to show how to use notebook_plotter() with a real dataset. However, real data would require downloading a dataset and extracting a spectrum which is beyond the scope of this guide. Instead, we will simulate a spectrum and save it locally as a dataset.  

#### Users who wish to learn more are about extracting spectra for real datasets are encouraged to see the [CIAO thread for point source extraction](https://cxc.cfa.harvard.edu/ciao/threads/pointlike/). 

#### In order to simulate a spectrum we will first need to download Chandra ARF and RMF response files for our instrument of choice (ACIS-S). Response files are necessary to convert an unconvolved astrophysical model to a convolved model which accounts for various instrumental effects such as effective area and detector resolution. In other words, the convolved model is what the telescope 'sees' if it is looking at a source with emission dictated by the  unconvolved astrophysical model.

#### For this example, we will use [recent Chandra responses generated for proposal planning purposes only](https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/). Please note that these responses are only meant to be used for simulating spectra and **should not be used** to analyze real Chandra observations. For real data, Chandra response files are generated specific to each observation.

In [ ]:
from urllib.request import urlretrieve

#download the ACIS-S ARF
url_arf = ('https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/CY26/aciss_aimpt_cy26.arf')
filename_arf = 'aciss_aimpt_cy26.arf'
urlretrieve(url_arf, filename_arf)

#download the ACIS-S RMF
url_rmf = ('https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/CY26/aciss_aimpt_cy26.rmf')
filename_rmf = 'aciss_aimpt_cy26.rmf'
urlretrieve(url_rmf, filename_rmf)

#### Now we can define a model, set some parameter values and simulate a spectrum using the sherpa function [fake_pha()](https://cxc.cfa.harvard.edu/sherpa/threads/fake_pha/). The model here is an absorbed, one-temperature thermal plasma typical of coronal emission from a bright, young pre-main sequence star.

In [ ]:
clean()

#lets define a model and set it as the source_model
model_for_simulation = xstbabs.asim * (xsvapec.psim)
set_source(model_for_simulation)

#here we can set some model parameters and the simulation exposure time
asim.nH = 0.1 
psim.kT = 2 #keV
psim.norm  = 1E-3

simulation_exposure = 150E3 # 150 ks

#simulate the spectrum and save it as a file in order to replicate the experience of loading in a real sources spectrum.
fake_pha(1, arf = filename_arf, rmf = filename_rmf, exposure = simulation_exposure)
save_pha(1,'simulated_spectrum.pi', clobber = 'yes')

model_for_simulation #lets print the model parameters to the screen

#### Now that we have a (simulated) dataset, lets load it in to sherpa as though it is a real source's spectrum. Note, the ARF and RMF should be automatically identified with the load_pha() command as those were the responses used to generate the spectrum.

In [ ]:
clean() #here we run clean() so the simulated dataset and source model are removed from the sherpa session.

pha_file = 'simulated_spectrum.pi'

load_pha(pha_file)

plot_data()

#### We can group the spectrum and plot it using the [plot_data()](https://cxc.cfa.harvard.edu/sherpa/threads/setplot_manual/) sherpa command.

In [ ]:
# ignore everthing but 0.5-8.0 keV counts and then group the spectrum
notice()
ignore(hi=0.5)
ignore(lo=8)
group_counts(1, 10)

#replot the grouped spectra with log yscale
plot_data(ylog=True)

#### Now that we have some data loaded, we will want to define a spectral model before we fit the data. This time lets use the xspec vapec model which allows us to change the plasma abundance values.

In [ ]:
#lets define a model and set it as the source_model
test_model_2 = xstbabs.ma2 * (xsvapec.p2)
set_source(test_model_2)
test_model_2

#### Since we are planning to fit simulated (faked) data with a model, we happen to know the exact model parameter values that generated the data. This would not be the case for a real observation. That being said, we can still use notebook_plotter() to set some reasonable parameter values before running fit(). After running the below notebook cell, try modifying the parameters **ma2.nH**, **p2.kT** and **p2.norm** until the convolved model (orange) looks more like the dataset. You may have to change the **p2.norm** parameter by a large amount. In addition to using the sliders, you can also click the values to the right of the sliders and set them with keyboard input. Don't forget that modifying the parameter values using the widgets will update the model parameter values. Therefore, running the fit() command after interacting with notebook_plotter will start using the values interactively set with notebook_plotter().

In [ ]:
ma2.nH.max = 30 #lets set nH to a physically-reasonable nH for nearby young stars

notebook_plotter(test_model_2, log_axis = 'ylog', autoscale = 'y', figsize_y = 3, plot_type = 'data_and_convolved_model')

#### The actual values used to simulate the dataset are **ma2.nH = 0.1 atoms $\mathrm{cm^{-2}}$**, **p2.kT = 2.0 keV**, and **p2.norm = 1E-3**. Try these in the plot above and see how the data and model change as you approach these 'correct' values. Also note that sherpa includes the [guess()](https://cxc.cfa.harvard.edu/sherpa/ahelp/guess.html) function for estimating parameter ranges given a loaded dataset.

#### Now let's fit() the data and overplot the best-fit model.

In [ ]:
#lets fit the model to the loaded dataset and inspect the data+fit
fit()

plot_fit(ylog=True)

#### Now that we've fit the data let's re-run notebook_plotter(). We will see below that the model parameters in sherpa have been set to the best-fit values determined in the fit. Note that previous jupyter notebook cells running notebook_plotter() will **not** be updated with these best-fit parameter values until you reload that cell. It is not recommend to run notebook_plotter() in multiple cells if you intend to be careful about setting model parameters like you would right before a fit.

In [ ]:
notebook_plotter(test_model_2, log_axis = 'ylog', autoscale = 'y', figsize_y = 3, plot_type = 'data_and_convolved_model')

#### Looking at an astrophysical model convolved with the detector response can sometimes 'hide' some details of the physics we may wish to visualize. With notebook_plotter we can view the contribution of the unconvolved model (purple) to the model convolved with the ACIS instrument responses (orange). As demonstrated in the case below, all of the emission lines in the unconvolved model are 'blurred' together due to the lower spectral resolution of ACIS CCD imaging. Try adjusting the model parameters to see how they affect the convolved model. This style of plot can be accessed by selecting plot_type = 'both_models' and is useful for identifying which model parameters are likely to affect the observed spectrum of a source.

In [ ]:
notebook_plotter(test_model_2, log_axis = 'ylog', figsize_y = 3, plot_type = 'both_models', autoscale = 'none', ylim_low = '1E-8', ylim_high = '1')

#### Note, when using plot_type = 'both_models', the plotting options such as autoscale, xy-limits and log_axis refer only to the unconvolved model (purple).  The y-axis of the convolved model (orange), which is plotting different y-axis units, will always autoscale to fit within the ymin and ymax ranges set by the notebook_plotter. Unlike the unconvolved model, the convolved model is affected by the response (e.g., the ARF and RMF) of the telescope.

#### You can also view this dataset in wavelength space with the [sherpa set_analysis()](https://cxc.cfa.harvard.edu/sherpa/ahelp/set_analysis.html) command. **Keep in mind this will set the analysis to 'wavelength' for the entire sherpa session or until you switch back using set_analysis('energy') or using the clean() function.** In wavelength space, the plot parameter 'resolution' value is still entered in units of keV and is automatically converted to angstrom for plotting.

In [ ]:
#We can also see what this spectra looks like in wavelength space by using set_analysis()
set_analysis('wavelength')

notebook_plotter(test_model_2, log_axis = 'ylog', figsize_y = 3, plot_type = 'both_models', autoscale = 'none', ylim_low = 1E-15, ylim_high = 10, xlim_low = 0, xlim_high = 20)

## **EXAMPLE 3 - Simulating a Chandra spectrum for a proposal with help from notebook_plotter()**



#### Let's say we're interested in submitting a [Chandra proposal](https://cxc.harvard.edu/proposer/) and we want to identify a sample of young, pre-main sequence stars whose spectrum may include a 6.4 keV emission feature. In young stars, a 6.4 keV emission feature typically comes from neutral iron fluorescence emission of cold circumstellar disk gas irradiated by coronal X-rays. It would help to first look at spectral models typically used to fit young stellar X-ray spectra to determine what characteristics of the spectrum are important for detecting an emission line at 6.4 keV.




#### We're going to use sherpa's fake_pha() command to simulate a spectrum but first we will need to choose a model. Let's go with a simple plasma model typically used to fit spectra of young stars. For the moment, we will choose a model that does **not** include the emission line of interest at 6.4 keV because most stars don't show this feature. We will add in this line component at a later step. In this example we will end up working with two sherpa datasets so we will explicitly set the dataset id parameters when calling the relevant sherpa functions.


In [ ]:
clean() #lets run clean() to remove the datasets of previous examples from our sherpa session.
test_model_3 = xstbabs.ma3 * xsapec.p3

#lets set some parameters typical of young stellar coronae
ma3.nH = 0.1
p3.kT = 1.3
p3.Abundanc = 0.5
p3.norm = 3E-3
test_model_3

#lets display the model parameters that were set
test_model_3

#### To fake a spectrum for a future proposal, we will need to use Chandra responses (ARF and RMF) relevant for the year of our proposed observations and for our instrument of choice. For this case, we will use the [same responses](https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/) in Example 2. **Please note that these responses are only meant to be used for simulating spectra and should not be used to analyze real Chandra observations.**

In [ ]:
from urllib.request import urlretrieve

#download the ACIS-S ARF
url_arf = ('https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/CY26/aciss_aimpt_cy26.arf')
filename_arf = 'aciss_aimpt_cy26.arf'
urlretrieve(url_arf, filename_arf)

#download the ACIS-S RMF
url_rmf = ('https://cxc.cfa.harvard.edu/caldb/prop_plan/imaging/CY26/aciss_aimpt_cy26.rmf')
filename_rmf = 'aciss_aimpt_cy26.rmf'
urlretrieve(url_rmf, filename_rmf)

#### Now lets use fake_pha() to simulate a typical young stellar coronal source in a 100 ks observation. We will explicitly assign dataset id to 1 because later we will work with another dataset id.

In [ ]:
set_source(1, test_model_3)

exposure_time = 100E3 # 100 ks

fake_pha(1, arf=filename_arf, rmf=filename_rmf, exposure = exposure_time)

#### We can ignore everthing but 0.5-8.0 keV counts, group the spectrum, and display the faked data with the convolved model using the sherpa command plot_model()

In [ ]:
notice()
ignore(hi=0.5)
ignore(lo=8)
group_counts(1, 7)

plot_data(1, ylog=True)
plot_model(1, overplot=True)

#### Let's use notebook_plotter() to inspect the simulated (faked) spectrum and its underlying model.

In [ ]:
ma3.nH.min = 0.01
ma3.nH.max = 30
p3.kT.min = 0.2
p3.kT.max = 9

notebook_plotter(test_model_3, dataset_id = 1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'data_and_convolved_model', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### We can see above that there are not many counts in the spectrum >~ 5 keV which is going to make it difficult to detect any specific emission features in this part of the spectrum. What happens if you take the **p3.kT** widget slider and increase the plasma temperature? This appears to increase the number of counts in the convolved model at higher energy (>~ 5keV). Let's reset the plasma temperature to 1.3 keV and take a closer look how the unconvolved model affects the convolved model in the next cell.

In [ ]:
p3.kT = 1.3 #just in case you changed the value of p3.kT in the interactive widget, lets set it back to 1.3 keV for the purposes of this notebook.

notebook_plotter(test_model_3, dataset_id =1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'both_models', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### In the cell above we see the underlying plasma model (purple) used to simulate the spectrum (the faked data) and the same model convolved with the Chandra ACIS-S responses (orange). For emission line sources like young, pre-main sequence stars, the shape of the spectrum in low resolution spectra (i.e., CCD imaging-resolution) is driven in large part by all the underlying emission lines.

#### Try changing the plasma temperature parameter (**p3.kT**) in this interactive model and note how the shape of the convolved model (orange) changes. As you increase plasma temperature, emission lines at higher energies increase in strength relative to those at energies <~ 1 keV pushing up the high energy portion of the spectrum. Of note is the very bright series of lines at $\sim$ 6.7 keV associated with iron (Fe XXV). This 6.7 keV feature has been identified in the spectra of many stars and is typically used to demonstrate that the underlying plasma is indeed hot. You can see how if you reduce the plasma temperature, this 6.7 keV emission feature is reduced relative to the emission at lower energies (<~ 2 keV).

#### Now, lets take a look at this simulated spectrum but with a new model that includes a 6.4 keV gaussian line component meant to represent Fe fluorescence emission. We will set all the other parameters initially to the same values as our simulated spectrum but this time adding in a 6.4 keV gaussian component. Since we want to see what the new model looks like convolved with the ACIS response, **we will have to explicitly set the new model to dataset 1 using set_source()**.

In [ ]:
test_model_4 = xstbabs.ma4 * (xsapec.p4 + xsgaussian.g4)

ma4.nH = 0.1
p4.kT = 1.3
#p4.kT = 3.0
p4.Abundanc = 0.5
p4.norm = 3E-3
g4.sigma = 0.0
g4.norm = 5E-7
g4.LineE = 6.4

ma4.nH.min = 0.01
ma4.nH.max = 30
p4.kT.min = 0.3
p4.kT.max = 5

g4.LineE.min = 6.39
g4.LineE.max = 6.41
g4.norm.max = 1E-2
g4.sigma.max = 0.05

In [ ]:
set_source(1, test_model_4) 

notebook_plotter(test_model_4, dataset_id=1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'both_models', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)


#### In the cell above you can now see the presence of a gaussian line at 6.4 keV in the unconvolved model (purple) and its contribution to the convolved model (orange). Try increasing the **g4.norm** parameter by manually typing a higher value in the widget box to see how an increase in line flux affects the spectrum. Given the current model parameters, it appears you'll need a **g4.norm value >~ 1E-8** in order to identify any feature. If you wish to zoom-in on this line, you can set the xlim_low and xlim_high values in the plotting options.

#### Now lets use the plot_type 'data_and_convolved_model' option to look at our new convolved model overlaid on the previous faked spectrum which does not include a 6.4 keV component.

In [ ]:
#here we will reset the model values in case they were modified too drastically in the previous notebook_plotter() cell.
ma4.nH = 0.1
p4.kT = 1.3
p4.Abundanc = 0.5
p4.norm = 3E-3
g4.sigma = 0.0
g4.norm = 5E-7
g4.LineE = 6.4

notebook_plotter(test_model_4, dataset_id =1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'data_and_convolved_model', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### In the cell above we can see the 6.4 emission line component in the convolved model but there are relatively few counts in our faked spectrum >~ 5 keV which will make it difficult to detect without significantly more exposure time or a much brighter source. We saw in a previous cell that increasing the plasma temperature will increase the relative number of counts at higher energies for a set normalization. Try increasing the value of **p4.kT** to be 3.0 keV. Compared to the previous simulation, this new (hotter) model has a lot more flux > 5 keV although we now have to increase the value of **g4.norm** to ~3E-6 so it is visible above the continuum of the spectrum.

#### Since this science case is focused on detecting a 6.4 keV feature, it will be easier to detect such a feature with Chandra by focusing on young stars that already exhibit hot plasma (e.g., >~ 3 keV) as there will be more counts in the continuum region around where we expect this feature.

#### So now let's try simulating another spectrum with this gaussian component at 6.4 keV and the above model parameters. While you should be able to run fake_pha() using the model values set in the previous cell's notebook_plotter(), its safer to just reset the values here before simulating. We will assign this new spectrum to a new dataset 2.

In [ ]:
ma4.nH = 0.1
p4.kT = 3.0
p4.Abundanc = 0.5
p4.norm = 3E-3
g4.sigma = 0.0
g4.norm = 3E-6
g4.LineE = 6.4

set_source(2, test_model_4)

exposure_time = 100E3 #100 ks

fake_pha(2, arf=filename_arf, rmf=filename_rmf, exposure = exposure_time)

#### Like before, lets ignore everthing but 0.5-8.0 keV, group the spectrum, and display the faked data with the convolved model using the sherpa plot_data() and plot_model() commands.

In [ ]:
#We can ignore everthing but 0.5-8.0 keV, group the spectrum, and display the faked data with the convolved model using sherpa commands
notice_id(2)
ignore_id(2, hi=0.5)
ignore_id(2, lo=8)

group_counts(2, 10)

plot_data(2, ylog=True)
plot_model(2, overplot=True)

#### In this simulated spectrum, we can now identify both the 6.4 and 6.7 keV emission lines.  Let's open it in notebook_plotter() for a final view.

In [ ]:
notebook_plotter(test_model_4, dataset_id=2, log_axis = 'ylog', autoscale = 'none',  plot_type = 'data_and_convolved_model', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### Now let's say we want to compare the above dataset 2 and its associated test_model_4 (the simulated data and model which include a 6.4 keV emission component) with our previous dataset 1 and its associated test_model_3 (the simulated data and model with no 6.4 keV component) in a cell below.

#### **Anytime we want to generate a new interactive notebook_plotter window with data and a model (in this case it's dataset 1 with test_model_3), we need to make sure the model is first assigned to that data in sherpa using set_source(). In this case, we will do set_source(1, test_model_3). This ensures the proper responses are used when convolving data.**

In [ ]:
set_source(1, test_model_3)

notebook_plotter(test_model_3, dataset_id=1, log_axis = 'ylog', autoscale = 'none',  plot_type = 'data_and_convolved_model', xlim_low = 0.3, xlim_high = 8.0, ylim_low = 1E-7, ylim_high = 1)

#### Now that we have a simulation of a source where we can detect these lines, a next and final step (left to the reader) would be to explore the parameter space for this model and simulation. If [proposing](https://cxc.harvard.edu/proposer/) to observe a real source, you'll want to simulate spectra using values appropriate for the real object (e.g., flux, plasma temperature, median energy, nH). After modifying the relevant parameters in the model, you can re-simulate a spectrum for each source to determine the best instrument setup and exposure time request. The 6.4 keV emission feature discussed in this example is rare and is limited to young stars hosting circumstellar disks with reservoirs of neutral Fe. 

#### A helpful tool in determining what normalization parameter you can use for your simulation is the sherpa command [calc_energy_flux()](https://cxc.cfa.harvard.edu/sherpa/ahelp/calc_energy_flux.html). This allows you to evaluate your loaded model with an energy range to determine the flux. If you'd like to simulate a real source's spectrum, be sure to match the simulation's flux with your source of interest's flux (via the plasma normalization in this case).

#### If you found notebook_plotter() and/or this tutorial helpful (or not) please feel free to provide feedback at principe@space.mit.edu.